# 1. Preprocesamiento: lógica de limpieza

# Importación de matplotlib para visualizaciones

In [4]:
import matplotlib.pyplot as plt

# Importar librería

In [5]:
from datasets import load_dataset
import pandas as pd
import re

# Carga del dataset de Amazon Reviews

In [6]:
dataset = load_dataset("SetFit/amazon_reviews_multi_es", split="train")
df = dataset.to_pandas()

Repo card metadata block was not found. Setting CardData to empty.


# Muestra de trabajo (5.000 registros)

In [7]:
df_sample = df.sample(n=5000, random_state=42)

df_sample.shape

(5000, 4)

# Inspección inicial de los textos

In [8]:
df_sample['text'].iloc[0]

'Mini usb cable de carga defectuoso por lo cual se estropearon los usb de las baterias'

# Definición de la función clean_text

In [9]:
import re

def clean_text(text):
    text = text.lower()                          # convertir a minúsculas
    text = re.sub(r'<.*?>', '', text)            # eliminar etiquetas HTML
    text = re.sub(r'http\S+|www\S+', '', text)   # eliminar URLs
    text = re.sub(r'[^a-záéíóúñü\s]', '', text)  # eliminar signos y emojis
    text = re.sub(r'\s+', ' ', text).strip()     # eliminar espacios extra
    return text

# Prueba de la función clean_text

In [10]:
ejemplo = "¡Excelente servicio! 😍"
clean_text(ejemplo)

'excelente servicio'

# Aplicación de la función clean_text al dataset

In [11]:
df_sample['text_clean'] = df_sample['text'].apply(clean_text)

In [12]:
print("🔍 Distribución labels ORIGINALES (0-4):")
print(df_sample['label'].value_counts().sort_index())

print("\n" + "="*50)

# Función mapeo TERNARIO (incluye NEUTRO)
def map_label_ternary(label):
    if label in [0, 1]:
        return 'Negativo'
    elif label == 2:
        return 'Neutro'
    else:  # 3, 4
        return 'Positivo'

# Crear columna ternaria
df_sample['label_tern'] = df_sample['label'].apply(map_label_ternary)

print("\n✅ Distribución TERNARIA (ANTES balanceo):")
print(df_sample['label_tern'].value_counts())
print(f"\nForma dataset: {df_sample.shape}")


🔍 Distribución labels ORIGINALES (0-4):
label
0    1007
1     982
2    1003
3     987
4    1021
Name: count, dtype: int64


✅ Distribución TERNARIA (ANTES balanceo):
label_tern
Positivo    2008
Negativo    1989
Neutro      1003
Name: count, dtype: int64

Forma dataset: (5000, 6)


# INSTALACION DE LIBRERÍA OVERSAMPLING 

In [13]:
!pip install imbalanced-learn
print(" imbalanced-learn instalado")

Defaulting to user installation because normal site-packages is not writeable
 imbalanced-learn instalado


# BALANCEAR CLASES (Prioridad Negativos)

In [14]:
from imblearn.over_sampling import RandomOverSampler
import pandas as pd

# Preparar datos para oversampling
X = df_sample[['text_clean']]  # Textos limpios
y = df_sample['label_tern']    # Etiquetas ternarias

print("🔍 ANTES balanceo:")
print(y.value_counts())

# Oversampling - Duplica Neutro hasta igualar otros
ros = RandomOverSampler(random_state=42)
X_balanced, y_balanced = ros.fit_resample(X, y)

# Nuevo dataframe balanceado
df_balanced = pd.DataFrame({
    'textclean': X_balanced['text_clean'], 
    'labeltern': y_balanced
})

print("\n DESPUÉS balanceo:")
print(df_balanced['labeltern'].value_counts())
print(f"Nuevo dataset: {df_balanced.shape}")

🔍 ANTES balanceo:
label_tern
Positivo    2008
Negativo    1989
Neutro      1003
Name: count, dtype: int64

 DESPUÉS balanceo:
labeltern
Neutro      2008
Negativo    2008
Positivo    2008
Name: count, dtype: int64
Nuevo dataset: (6024, 2)


# GUARDAR DATASET TERNARIO BALANCEADO

In [15]:
df_balanced[['textclean', 'labeltern']].to_csv(
    'amazon_reviews_es_5000_balanced.csv', 
    index=False
)
print(" GUARDADO FINAL: amazon_reviews_es_5000_balanced.csv")
print(" Dataset listo para 03_training.ipynb!")
print("\nPrimeras 3 filas:")
print(df_balanced.head(3))
print(f"\n SHAPE FINAL: {df_balanced.shape}")

 GUARDADO FINAL: amazon_reviews_es_5000_balanced.csv
 Dataset listo para 03_training.ipynb!

Primeras 3 filas:
                                           textclean labeltern
0  mini usb cable de carga defectuoso por lo cual...    Neutro
1  el cable es muy largo y es comodo pero me duró...  Negativo
2                         ideal para mis necesidades  Positivo

 SHAPE FINAL: (6024, 2)


# Verificación antes y después de la limpieza

In [16]:
df_sample[['text', 'text_clean']].head(5)

,text,text_clean
119737,Mini usb cable de carga defectuoso por lo cual...,mini usb cable de carga defectuoso por lo cual...
72272,El cable es muy largo y es comodo pero me duró...,el cable es muy largo y es comodo pero me duró...
158154,Ideal para mis necesidades.,ideal para mis necesidades
65426,"Es desigual el color, y mancha. El hilo de rom...",es desigual el color y mancha el hilo de rompe...
30074,Los comentarios se hacen cuando uno recibe la ...,los comentarios se hacen cuando uno recibe la ...


# DATASET FINAL: TERNARIO BALANCEADO

Este dataset:
- Tiene 3 clases bien definidas: Negativo (0-1 estrellas), Neutro (2 estrellas), Positivo (3-4 estrellas)
- Está perfectamente balanceado (2008 ejemplos por clase)
- Prioriza la correcta identificación de comentarios negativos 
- Es el que debes usar para entrenar el modelo en 03_training.ipynb

In [26]:
# Usamos el df_balanced que ya creamos con oversampling
df_final = df_balanced.copy()

# Renombramos columnas para mayor claridad y consistencia con el pipeline de entrenamiento
df_final.rename(columns={
    'textclean': 'text_clean',
    'labeltern': 'label'
}, inplace=True)

# Verificación final
print("Distribución de clases en el dataset final (debe estar balanceado):")
print(df_final['label'].value_counts().sort_values(ascending=False))

print(f"\nShape del dataset final: {df_final.shape}")
print("\nPrimeras 5 filas:")
display(df_final.head())

print("\nÚltimas 5 filas (para verificar variedad):")
display(df_final.tail())

Distribución de clases en el dataset final (debe estar balanceado):
label
Neutro      2008
Negativo    2008
Positivo    2008
Name: count, dtype: int64

Shape del dataset final: (6024, 2)

Primeras 5 filas:


,text_clean,label
0,mini usb cable de carga defectuoso por lo cual...,Neutro
1,el cable es muy largo y es comodo pero me duró...,Negativo
2,ideal para mis necesidades,Positivo
3,es desigual el color y mancha el hilo de rompe...,Negativo
4,los comentarios se hacen cuando uno recibe la ...,Negativo



Últimas 5 filas (para verificar variedad):


,text_clean,label
6019,la verdad es que los dos que he puesto se han ...,Neutro
6020,el agarrador parece que se va a romper con sol...,Neutro
6021,dura mucho pero el color nude es transparente ...,Neutro
6022,fácil de montar bonito y ligero lo único malo ...,Neutro
6023,es como imaginaba material muy finito solo sir...,Neutro


In [29]:
# GUARDAR EL DATASET PARA ENTRENAMIENTO
output_path = '../data/processed/amazon_reviews_es_ternario_balanced.csv'

df_final[['text_clean', 'label']].to_csv(
    output_path,
    index=False,
    encoding='utf-8-sig'
)

print(f"\n¡DATASET FINAL GUARDADO CORRECTAMENTE!")
print(f"   Ruta: {output_path}")
print(f"   Filas: {df_final.shape[0]} | Clases: {sorted(df_final['label'].unique())}")
print("   ds/data/processed/amazon_reviews_es_ternario_balanced.csv")


¡DATASET FINAL GUARDADO CORRECTAMENTE!
   Ruta: ../data/processed/amazon_reviews_es_ternario_balanced.csv
   Filas: 6024 | Clases: ['Negativo', 'Neutro', 'Positivo']
   ds/data/processed/amazon_reviews_es_ternario_balanced.csv
